In [1]:
import pandas as pd
import numpy as np

from unidecode import unidecode

import requests

import folium

## Prepare data for map
We start retrieving the data from the previous notebook, treating the data for visualization and creating some auxiliary functions

In [2]:
neighborhood_df = pd.read_csv('../data/processed/neighbourhoods.csv')

In [3]:
# load the GeoJSON data and use 'UTF-8'encoding
geojson_natal_file = requests.get('https://github.com/nymarya/data-science-one/blob/master/Lesson%2314/natal.geojson?raw=true')
geo_json_natal = geojson_natal_file.json()

In [4]:
# insert additional information ('population', 'income') into GeoJSON file
# we do this for showing in the popup
for pop in geo_json_natal['features']:
    codarea = pop['properties']['name']
    codarea = codarea if codarea != 'Felipe Camarão' else 'Filipe Camarão'
    neigh = neighborhood_df.query('name_norm == "' + unidecode(codarea) + '"')
    pop['properties']['population'] = str(neigh.inhabitants.values[0])
    pop['properties']['income'] = str(neigh.income.values[0])
    pop['properties']['name'] = str(neigh.name.values[0]) # assure the name is the same

In [5]:
def build_map(df_key, geojson_key, legend):
    """ Function for creating the map.
    Attributes
    ---------
    df_key: str
        key name in dataframe. Ex: inhabitant
    geojson_key: list
        key name in geojson. Ex: population
    legend: str
        word to be shown in legend. Ex:population .
    """
    # Create a map object
    m = folium.Map(
        location=[-5.826592, -35.212558], # Natal
        zoom_start=11.3,
        tiles='Stamen Terrain'
    )

    # create a threshold of legend
    bins = np.linspace(neighborhood_df[df_key].min(),
                       neighborhood_df[df_key].max(),
                       6).tolist()
    choropleth = folium.Choropleth(
        geo_data=geo_json_natal,
        data=neighborhood_df,
        name= "neighborhoods",
        columns=['name', df_key],
        key_on='feature.properties.name',
        fill_color='BuPu',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name='#' + legend+' in Natal-RN',
        bins=bins
    ).add_to(m)


    folium.LayerControl().add_to(m)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(fields=['name',geojson_key],
                                       aliases=['Name:',legend.capitalize() + ":"],
                                       localize=True)
    )
    return m

## Basic maps

We start by plotting population and income distribution. Along the notebook, all maps will be saved in a dict to be exported as HTML at the end.

In [6]:
maps = {}

### Population

Up to the north, we see that there is a concentration of populous districts

In [7]:
maps['population'] = build_map('inhabitants', 'population', 'population')
maps['population']

### Income

Although in the north of the city there is more people, the income is concentrated in the southerns and easterns districts 

In [8]:
maps['income'] = build_map('income', 'income', 'income')
maps['income']

## Pedestrian accessibility

From now on, we will plot the information gathered in the previous notebook. We start by adding the information to the GeoJSON again.

In [9]:
for pop in geo_json_natal['features']:
    codarea = pop['properties']['name']
    codarea = codarea if codarea != 'Felipe Camarão' else 'Filipe Camarão'
    neigh = neighborhood_df.query('name_norm == "' + unidecode(codarea) + '"')
    pop['properties']['dist_school'] = str(neigh.nearest_school.values[0])
    pop['properties']['dist_hospital'] = str(neigh.nearest_hospital.values[0])
    pop['properties']['dist_police'] = str(neigh.nearest_police.values[0])
    pop['properties']['dist_restaurant'] = str(neigh.nearest_restaurant.values[0])
    pop['properties']['dist_bank'] = str(neigh.nearest_bank.values[0])
    pop['properties']['dist_pharmacy'] = str(neigh.nearest_pharmacy.values[0])

### Schools
People living in the districts in north and west have to walk more to get to a school

In [10]:
maps['school'] = build_map('nearest_school', 'dist_school', 'distance from center to nearest school')
maps['school']

### Hospitals

Again, people have a harder time getting to a hospital. Some districts in the south and west have to walk at least 2km to get to a hospital

In [11]:
maps['hospital'] = build_map('nearest_hospital', 'dist_hospital', 'distance from center to nearest hospital')
maps['hospital']

### Police stations

We have a situation similar to schools, in some districts we hit the mark of 10km

In [12]:
maps['police'] = build_map('nearest_police', 'dist_police', 'distance from center to nearest police')
maps['police']

### Restaurants
There are fewer district colored in purple/lilas this time. Also, the max distance is now 5km, but there is still struggle in districts like Lagoa Azul, Nossa Senhora da Apresentação and Guarapés

In [13]:
maps['restaurant'] = build_map('nearest_restaurant', 'dist_restaurant', 
                               'distance from center to nearest restaurant')
maps['restaurant']

### Bank
For the first time, Nossa Senhora da Apresentação and Lagoa Azul aren't the districts with the darker color. When it comes to banks, people in the west of the city have to walk more to find one.

In [14]:
maps['bank'] = build_map('nearest_bank', 'dist_bank', 'distance from center to nearest bank')
maps['bank']

### Pharmacy

In [15]:
maps['pharmacy'] = build_map('nearest_pharmacy', 'dist_pharmacy', 
                             'distance from center to nearest pharmacy')
maps['pharmacy']

In [17]:
names = ['population', 'income', 'hospital', 'restaurant', 'bank','school', 'police', 'pharmacy']
for name in names:
    maps[name].save(f'../reports/{name}.html')